In [1]:
from pyspark.sql import SparkSession, Row
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

spark = SparkSession.builder.master("local").appName("spark-persistence").getOrCreate()

24/08/12 15:22:49 WARN Utils: Your hostname, MZC01-HYUCKSANGCHO.local resolves to a loopback address: 127.0.0.1; using 10.90.11.150 instead (on interface en0)
24/08/12 15:22:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/12 15:22:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/12 15:22:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### CSV & JSON

In [9]:
dfListing = spark.read.load("./airbnb_data/airbnb_listings.csv",
                        format="csv", inferSchema=True, header=True,
                        sep=",", quote='"',escape ='"',multiline=True)

dfListing.show()

+------+--------------------+--------------+------------+--------------------+--------------------+--------------------+--------------------+-------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+----------+--------------------+--------------+-------+--------------------+-------------------+----------+--------------------+--------------------+------------------+------------------+--------------------+-----------------+--------------------+--------------------+------------------+-------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+---------------+----------------------+----------------------------+---------+-----+-------+------+--------------+------------+-------------+--------+----------+-----------------+-------------+---------------+------------+---------+--------+----+--------+--------------------+---

In [10]:
dfListing.count()

4865

In [22]:
dfListingSelected = dfListing.selectExpr(
    "id as listing_id",
    "listing_url", 
    "name as listing_name", 
    "summary as listing_summary", 
    "description as listing_desc"
)

dfListingSelected\
    .repartition(2)\
    .write\
    .mode("overwrite")\
    .format("json")\
    .option("compression", "gzip")\
    .save("airbnb_listings")

In [30]:
dfListingJson = spark.read.format("json").load("./airbnb_listings")

dfListingJson.show(truncate=True)

+--------------------+----------+--------------------+--------------------+--------------------+
|        listing_desc|listing_id|        listing_name|     listing_summary|         listing_url|
+--------------------+----------+--------------------+--------------------+--------------------+
|Get ready to kick...|  24180313|Capitol Hill Embe...|Get ready to kick...|https://www.airbn...|
|Fun, colorful, re...|  36346977|Entrepreneur's Pa...|Fun, colorful, re...|https://www.airbn...|
|Brand New Loft ap...|  30312243|Beautiful Sunny S...|Brand New Loft ap...|https://www.airbn...|
|Unwind and enjoy ...|  39116323|Historic N. Park ...|Unwind and enjoy ...|https://www.airbn...|
|My place is close...|  16944159|Great New Apartme...|My place is close...|https://www.airbn...|
|Step back to the ...|  20169840|Beautiful Denver ...|Step back to the ...|https://www.airbn...|
|Astounding townho...|  32446997|Luxurious LoHi To...|Astounding townho...|https://www.airbn...|
|Well established ...|  111230

In [32]:
dfListingSelected\
    .repartition(1)\
    .write\
    .mode("overwrite")\
    .format("csv")\
    .option("header", "true")\
    .save("airbnb_listings_selected")

### Parquet
Parquet, ORC같은 Columnar Foramt이 필요한 이유는
- Column Pruning : 대부분의 경우 몇몇 컬럼만 지정하여 조회하기 때문에 컬럼 기준인 경우 IO 비용을 줄 이 수 있다.
- Column Encoding : 타입에 특화된 인코딩을 통해 저장공간을 줄 일 수 있다.
- Predicate PushDown : Spark로 가져오기 전에 미리 필터링하여 가져오기 때문에 네트워크 비용을 줄 일 수 있다.
- 백터화된 연산이 가능하다.

In [35]:
dfListingSelected\
    .repartition(1)\
    .write\
    .mode("error")\
    .format("parquet")\
    .option("compression", "snappy")\
    .save("airbnb_listings_parquet")

In [36]:
dfListingParquet = spark.read.format("parquet").load("./airbnb_listings_parquet")

dfListingParquet.show()

+----------+--------------------+--------------------+--------------------+--------------------+
|listing_id|         listing_url|        listing_name|     listing_summary|        listing_desc|
+----------+--------------------+--------------------+--------------------+--------------------+
|       360|https://www.airbn...|LoHi Secret garde...|Come enjoy our oa...|Come enjoy our oa...|
|       590|https://www.airbn...|Comfortable  - an...|Large guest room ...|Large guest room ...|
|       592|https://www.airbn...|             private|This room is in t...|This room is in t...|
|      1940|https://www.airbn...|Baker Studio Clos...|Great place for a...|Great place for a...|
|      2086|https://www.airbn...|  Garden Level Condo|A furnished, gard...|A furnished, gard...|
|     31503|https://www.airbn...|Highland Park Gue...|                null|Highland Park Gue...|
|     39405|https://www.airbn...|LoHi Secret garde...|Come enjoy our oa...|Come enjoy our oa...|
|     56185|https://www.airbn.

In [37]:
dfListingParquet.rdd.getNumPartitions()

1

In [38]:
dfListingParquet.printSchema()

root
 |-- listing_id: integer (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- listing_name: string (nullable = true)
 |-- listing_summary: string (nullable = true)
 |-- listing_desc: string (nullable = true)



In [39]:
# spark.read 로 파일을 읽을 경우에는 파일 사이즈나 숫자에 의해 동적으로 Partition 숫자가 spark.default.parallelism (default = 200) 값에 따라 조절됩니다. 
dfListingSelected\
    .repartition(2000)\
    .write\
    .mode("overwrite")\
    .format("parquet")\
    .option("compression", "snappy")\
    .save("airbnb_listings_parquet")

In [41]:
dfListingParquet = spark.read.format("parquet").load("./airbnb_listings_parquet")


In [45]:
dfListingParquet.rdd.getNumPartitions()

63

### Avro - Data Serialization
Avro 타입은 Data Serialization(직렬화)도구 입니다.  
JSON으로 데이터를 전송하는 것과 달리 Avro, Protobuf, Thrift와 같은 언어 중립적인 데이터 직렬화 포맷을 사용하는 이유가 있습니다.  
1. 다양한 언어로 데이터를 읽고 쓸 수 있습니다.
2. 스키마를 미리 정의하기 때문에 실수로 변경되는 것을 방지합니다.
3. 타입이 정해져 있으므로 Encoding을 통한 사이즈 축소 및 효율적인 조회가 가능합니다.

Column 포맷은 주로 대규모 데이터를 읽고 쓰는 경우에 사용하고, Kafka/API 등 실시간으로 데이터를 주고 받는 경우에는 Avro와 같은 Schema관리가 편한 포맷을 사용합니다.  


In [46]:
schemaAvroListingV1 = """
{
    "type": "record",
    "name": "AirbnbListing",
    "namespace": "com.airbnb",
    "fields": [
    {"name": "listing_id", "type": "int"},
    {"name": "listing_url", "type": "string"},
    {"name": "listing_name", "type": "string"},
    {"name": "listing_summary", "type": ["string", "null"]},
    {"name": "listing_desc", "type": ["string", "null"]}
  ]
}
"""

dfListingSelected\
    .repartition(2)\
    .write\
    .mode("overwrite")\
    .format("avro")\
    .option("avroSchema", schemaAvroListingV1)\
    .save("airbnb_listings_avro")

AnalysisException: Failed to find data source: avro. Avro is built-in but external data source module since Spark 2.4. Please deploy the application as per the deployment section of Apache Avro Data Source Guide.